In [ ]:
import numpy as np, matplotlib.pyplot as plt, json, pickle, scipy.sparse, sys, scipy.sparse as ss
from helper_functions_py3 import *
from sklearn.preprocessing import normalize
%matplotlib inline

## Load data

In [86]:
E = ss.load_npz('counts_norm.npz')
lk_lsk = np.load('lk_lsk.npy')
tps_flat = np.load('tps_flat.npy')
gene_list = load_genes('genes.txt')

In [87]:
fate_names = open('FINAL_fate_names.txt').read().split('\n')
BMq = np.load('BMq.npy')
mpp_ff = np.array([x.decode('utf-8') for x in lk_lsk[tps_flat==2]])=='LSK'
BM_ff = np.all([BMq.sum(1)>0, mpp_ff],axis=0)
Y = BMq[BM_ff,:] / BMq[BM_ff,:].sum(1)[:,None]


In [88]:
E2 = E[tps_flat==2,:]

In [89]:
gf = filter_genes(E2[mpp_ff,:],3,3,70)

In [90]:
TFs = [l.split('\t')[2] for l in open('tf_list.txt').read().split('\n')]
TFs = np.array([gene_list.index(g) for g in TFs if g in gene_list])

In [93]:
DFE = [l.split('\t')[1] for l in open('enriched_genes_IN_VITRO.txt').read().split('\n')]
DFE = np.array([gene_list.index(g) for g in DFE])

In [94]:
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPRegressor, MLPClassifier
from scipy.stats import pearsonr

### choose best hyper for LR

In [ ]:
all_accs = []
for i,gene_ff in enumerate([[],TFs, DFE, gf]):
    if len(gene_ff)==0: gene_ff = np.random.randint(0,len(gene_list),len(TFs))
    EE = E2[BM_ff,:][:,gene_ff]
        
    accs = []
    rr = np.random.uniform(0,1,BM_ff.sum())
    train = rr < .5
    test = rr > .5
    for jj,C in enumerate(np.logspace(-3,3,14)):
        LR = LogisticRegression(C=C)
        LR.fit(EE[train,:],np.argmax(Y[train,:],axis=1))
        YY = LR.predict(EE[test,:])
        accs.append(np.mean(np.argmax(Y[test,:],axis=1)==YY))
    all_accs.append(accs)

In [ ]:
for acc in all_accs:
    plt.plot(acc)

In [ ]:
for acc in all_accs:
    plt.plot(acc)
plt.ylabel('Accuracy')
plt.xlabel('Log regularization (C)')
plt.plot([2]*2,[0.35,.62],'--k')
plt.xticks(range(14)[::2], [round(x,1) for x in np.linspace(-3,3,14)][::2]);
plt.gcf().set_size_inches((4,3))
plt.savefig('RG_reg_scan.pdf')

### choose best params for MLP

In [ ]:
rr = np.random.uniform(0,1,BM_ff.sum())
test = np.random.uniform(0,1,BM_ff.sum()) > .5
train = np.logical_not(test)

all_accs = []
all_layer_nums = [(100,20)] #(100,50 ,20),(200,50 ,20), (50 ,50 ,20),(200,100,20),(200,50), 
                  #(200,100),(200,20),(100,50),(100,20),(200,100,50,20)]
for i,gene_ff in enumerate([[],TFs, DFE, gf]):
    if len(gene_ff)==0: gene_ff = np.random.randint(0,len(gene_list),len(TFs))
    EE = E2[BM_ff,:][:,gene_ff]
    accs = np.zeros((10,8))
    for ii,layer_nums in enumerate(all_layer_nums):
        for jj,alpha in enumerate(np.logspace(-3,1,8)): 
            print(ii,jj)
            MLP = MLPClassifier(hidden_layer_sizes=layer_nums, max_iter=100000, alpha=alpha)
            MLP.fit(EE[train,:],np.argmax(Y[train,:],axis=1))
            YY = MLP.predict(EE[test,:])
            accs[ii,jj] = np.mean(np.argmax(Y[test,:],axis=1)==YY)
    
    all_accs.append(accs)

In [ ]:
all_layer_nums = [(100,50 ,20),(200,50 ,20), (50 ,50 ,20),(200,100,20),(200,50), 
                  (200,100),(200,20),(100,50),(100,20),(200,100,50,20)]

In [ ]:
all_accs = np.array(all_accs)

In [ ]:
for i in range(4):
    plt.plot(all_accs[i][0,2:])
plt.xticks(range(6), [round(x,2) for x in np.linspace(-3,1,8)[2:]]);
plt.plot([3]*2,[0.38,.57],'--k')
plt.xlabel('Log regularization (alpha)')
plt.ylabel('Accuracy')
plt.gcf().set_size_inches((4,3))
plt.savefig('NN_reg_scan.pdf')

In [ ]:
acc = np.zeros((100,4,2))
for i,gene_ff in enumerate([[],TFs, DFE, gf]): #TFs, gf, np.ones(len(gene_list))>0]):
    if len(gene_ff)==0: gene_ff = np.random.randint(0,len(gene_list),len(TFs))
    EE = E2[BM_ff,:][:,gene_ff].toarray()
    print(i)
    #if i < 3: continue
    for ii in range(100):

        rr = np.random.uniform(0,1,BM_ff.sum())
        train = rr < .75
        test = rr > .75
        
        LR = LogisticRegression(C= 8.37677640e-03, max_iter=100)
        LR.fit(EE[train,:],np.argmax(Y[train,:],axis=1))
        YY = LR.predict(EE[test,:])
        acc[ii,i,0] = np.mean(np.argmax(Y[test,:],axis=1)==YY)

        MLP = MLPClassifier(hidden_layer_sizes=(100, 20), alpha=.01, activation='relu', solver='adam', momentum=0.9)
        MLP.fit(EE[train,:],np.argmax(Y[train,:],axis=1))
        YY = MLP.predict(EE[test,:])
        acc[ii,i,1] = np.mean(np.argmax(Y[test,:],axis=1)==YY)
        
        print(acc[ii,i,:])
        
        
    

In [ ]:
X = np.mean(acc,axis=0)
Xerr = np.std(acc,axis=0)
plt.errorbar(range(4),X[:,0],yerr=Xerr[:,0]/2,fmt='o', capsize=2, ms=4)
plt.errorbar(np.arange(4)+.2,X[:,1],yerr=Xerr[:,1]/2,fmt='o', capsize=2, ms=4)
plt.xticks(range(4))
plt.xlim([-.25,3.35])
plt.ylim([.38,0.64])
plt.gcf().set_size_inches((2.5*.9,2*.9))

In [ ]:
from scipy.stats import mannwhitneyu, ttest_ind
mm = []
for i in range(4):
    print(mannwhitneyu(acc[:,i,0],acc[:,i,1])[1])
    mm.append(acc[:,i,1].mean()/acc[:,i,0].mean()-1)
print(np.mean(mm))

In [ ]:
for i in range(3):
    print(mannwhitneyu(acc[:,i,:].flatten(),acc[:,i+1,:].flatten())[1])
